In [ ]:
"""
h5形式のモデルをSavedModelへ変換し、SageMakerのエンドポイントへデプロイする
"""

import boto3
import json
import os 
 
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow.serving import Model
 
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import model_from_json, load_model
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
 
# IAMロール定義取得
role = get_execution_role()

In [ ]:
# h5形式のモデル読み込み
model = load_model('model.h5')

# SavedModel形式への変換
input_sig  = 'inputs'
output_sig = 'score'
signature = predict_signature_def(inputs={input_sig: model.input}, outputs={output_sig:model.output})
model_version = '1'
export_path   = './tf_model/mnist/' + model_version
if not os.path.exists(export_path):
    os.mkdir(export_path)

with K.get_session() as sess:
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
    builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING], signature_def_map={'serving_default': signature}, main_op=tf.tables_initializer())
    builder.save()

In [ ]:
# サーバ側の推論用ソースコードを含めた形でSavedModel形式のモデルを圧縮
!cd /home/ec2-user/SageMaker
! tar zcvf model.tar.gz  tf_model/mnist tf_model/code

In [ ]:
# S3へモデルデータをアップロード
sagemaker_session  = sagemaker.Session()
model_data = sagemaker_session.upload_data(path='tf_model/model.tar.gz', key_prefix='model')
print("done")

In [ ]:
# サーバーレス設定
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=4096,
    max_concurrency=3
)

# エンドポイントへモデルをデプロイ
tensorflow_serving_mnist_model = Model(model_data=model_data,
                                        role=role,
                                        framework_version='1.15.5')
predictor = tensorflow_serving_mnist_model.deploy(initial_instance_count=1,
                                                    instance_type='ml.t2.medium',
                                                    endpoint_name='mnist-endpoint',
                                                    serverless_inference_config=serverless_config)

In [ ]:
# 推論の動作確認
import pickle as pkl
img = pkl.load(open("data/mnist.pickle", "rb"))
len(img[0][0]), len(img[0][1])
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_train /= 255
client = boto3.client("sagemaker-runtime")
res = client.invoke_endpoint(EndpointName=predictor.endpoint_name,
                            Body=json.dumps({'inputs':[x_train[0].tolist()]}),
                            ContentType='application/json',
                            Accept='application/json')
result = json.loads(res['Body'].read())

In [ ]:
result